In [1]:
import json

with open('/home/ubuntu/server2/ssd3/gpt4all/translated-gpt4all.json') as fopen:
    gpt4all = json.load(fopen)

In [2]:
import random

malay_respond = [
    ' respond in malay',
    ' reply in malay',
    ' reply bahasa melayu',
    ' ckp bahasa melayu',
    ''
]

In [3]:
template = {
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}

In [4]:
import malaya

model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3', force_check = False)

In [5]:
_ = model.cuda()

In [6]:
model.generate(['bagaimana saya boleh kurangkan berat badan tanpa pergi ke gim?'],
              max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['mcm mana aku nak kurangkan berat badan x pegi gym',
 'macam mana aku boleh kurangkan berat badan tanpa pergi gym tu',
 'Macam mana nak kurangkan berat badan tanpa pergi gym',
 'macam mana aku nak kurangkan berat badan tanpa gi gym',
 'bagaimana aku boleh kurangkan berat badan tanpa pergi gym']

In [13]:
from tqdm import tqdm

data = []
for d in tqdm(gpt4all):
    if 'prompt' not in d or 'response' not in d:
        continue
    
    try:
        if random.random() > 0.5:
            q = d['prompt'][0] + random.choice(malay_respond)
        else:
            q = d['prompt'][1]
            if random.random() > 0.5 and len(q.split()) < 20:
                generated = model.generate(
                    [q],
                    max_length = 100, 
                    do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                    num_return_sequences=5,
                )
                q = random.choice(generated)
            
        if random.random() > 0.5:
            q = q.lower()

        a = d['response'][-1]
        res = template["prompt_no_input"].format(instruction=q)
        res = f"{res}{a}"
        data.append(res)
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████████████| 437605/437605 [1:14:57<00:00, 97.31it/s]


In [14]:
len(data)

332550

In [15]:
data

['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\napakah bentuk perjalanan yang paling selamat?\n\n### Response:\nMenurut statistik, mengembara dengan penerbangan komersial adalah bentuk pengangkutan yang paling selamat.',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the name of the band that sings "Stairway to Heaven"? ckp bahasa melayu\n\n### Response:\nKumpulan muzik yang menyanyikan "Stairway to Heaven" ialah Led Zeppelin.',
 "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nwhat was the name of the book that was written by gabriel harvey in 1593? reply bahasa melayu\n\n### Response:\nNama buku yang ditulis oleh Gabriel Harvey pada tahun 1593 ialah 'Four Letters and Certain Sonnets, Especially Touching Robert Greene'.",
 'Below is an ins

In [16]:
with open('prepared-gpt4all.json', 'w') as fopen:
    json.dump(data, fopen)